# Le PIB est-il un indicateur robuste de l'accès à l'alimentation ?

In [ ]:
pip install faostat

In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Partie 1 : Collecte des données

### Récupération des données de PIB à partir de la base de données de la banque mondiale

In [ ]:
url = "https://api.worldbank.org/v2/country/all/indicator/NY.GDP.PCAP.KD?format=json&per_page=20000"


response = requests.get(url)

# Extraction du JSON de la réponse
data = response.json()  # ici c’est correct

# Vérification des données
print(type(data),len(data))
print(data[0]) # Métadonnées
print(data[1][:5]) # Données de PIB par pays

data = PIB_hab[1]  # On ne garde pas les métadonnées



#Retraitement des données


Création du DataFrame

In [ ]:
from import_worldbank import parse_worldbank_pib

In [ ]:
# On transforme le JSON en DataFrame et on ne garde que le PIB/hab pour chaque années avec le nom et le code du pays
data_worldbank = parse_worldbank_pib(data)
print(data_worldbank.head(10))

Filtrage des années à garder

In [ ]:
from import_worldbank import filter_years

In [ ]:
data_worldbank = filter_years(data_worldbank,2000,2020)
print(data_worldbank.head(10))

Restriction aux pays suivants : France, Etats-unis, Angleterre, Russie, Chine, Japon, Brésil, Afrique du Sud

In [ ]:
from import_worldbank import filter_bricsam_countries

In [ ]:
pays_a_conserver = [
        'India', 'China', 'Brazil', 'South Africa', 
        'Russian Federation', 'Mexico'
    ]

data_worldbank = filter_bricsam_countries(data_worldbank, pays_a_conserver)

# Vérifier le résultat
print(data_worldbank['Pays'].unique())
print(data_worldbank.head(10))

Gestion des valeurs manquantes

In [ ]:
for pays in data_worldbank['Pays'].unique() :
    print(data_worldbank.loc[data_worldbank['Pays'] == pays].isna().sum(),pays)

Export en csv

In [ ]:
data_worldbank.to_csv("données_worldbank_BRICSAM.csv")

### Récupération des données indicateurs alimentaires à partir de la base de données FAOSTAT

In [ ]:
import faostat

df = faostat.list_datasets_df() # cette fonction permet de récupérer l'ensemble des indicateurs disponible
                                # ainsi que d'avoir des informations sur les màj de versions, etc
print(df)

**On utilise la fonction définie dans le fichier py import des données faostat pour récupérer les données de 4 indicateurs  pour les pays des BRICSAM sur la période 2000 à 2022.**
**Les indicateurs sont :**
**"suffisance des apports énergétiques alimentaires moyens"**
**"disponibilité alimentaire par habitant"**
**"disponibilité protéiques moyenne"**
**"prévalence de la sous alimentation"**


In [ ]:
from import_fao import fetch_fao_security_data

In [ ]:

liste_pays = [202,185,138,100,351,21] # Correspond aux codes des pays des BRICSAM
liste_indicateurs = ['21010','22000','21013'] # Correspond aux codes des indicateurs d'insécurité alimentaire
liste_années = [2000, 2022] # Correspond à la page de dates.

data_faostat = fetch_fao_security_data(liste_pays, liste_indicateurs, liste_années)

print(data_faostat.head(10))



On vérifie ensuite qu'il n'y a pas de valeurs manquantes dans nos données. Comme ce n'est pas le cas, nous n'avons pas besoin de passer par un retraitement.

In [ ]:
print(data_faostat.isna().sum())

Une fois que l'on a récupéré les données qui nous intéressent et avoir vérifié qu'il n'y avait pas de valeurs manquantes dans nos données, on passe par un retraitement des données (renommage des colonnes, supression de certaines, etc.)

In [ ]:
from import_fao import clean_fao_columns

In [ ]:
data_faostat = clean_fao_columns(data_faostat)
print(data_faostat.head(10))


Ensuite on traduit en français le nom des indicateurs pour faciliter les manipulations par la suite

In [ ]:
from import_fao import translate_indicators

In [ ]:
data_faostat = translate_indicators(data_faostat)
print(data_faostat.head(10))


**Les indicateurs ont été moyennés sur 3 ans, et non par année. Nous avons des moyennes pour des périodes comme 2000-2002 et 2001-2003. Pour modéliser les données du PIB par habitant, nous avons décidé d'attribuer chaque moyenne triennale à la première année. Par exemple, la moyenne pour 2011-2013 correspond à 2011. Cela permet de maintenir la cohérence des tendances. Toutefois, si le PIB par habitant subit un choc au cours d'une année, le calcul de la moyenne peut atténuer ce choc sur les indicateurs alimentaires, ce qui affaiblit les résultats.**

In [ ]:
from import_fao import simplify_fao_years

In [ ]:
data_faostat = simplify_fao_years(data_faostat)
print(data_faostat.head(10))


Finalement, on exporte le dataframe en un fichier csv pour ne pas s'embeter à relancer tout le notebook à chaque fois

In [ ]:
data_faostat.to_csv("données_faostat_BRICSAM.csv")

## Partie 2 : DATA VISUALIZATION

In [ ]:
print(data_worldbank.head(10))

In [ ]:
print(data_faostat.head(10))

Evolutions des PIB/HAB

In [ ]:
for pays in data_worldbank['Pays'].unique() :
    PIB_hab_pays=PIB_hab_BRICSAM.loc[PIB_hab_BRICSAM['Pays'] == pays]
    plt.plot(PIB_hab_pays['Pays'],PIB_hab_pays['PIB_par_habitant'],label=pays)

plt.xlabel('Année')    
plt.ylabel('PIB par habitant ($ US courants)')
plt.grid()
plt.legend()
plt.xlim(left=2000, right=2025)
plt.show()

Illustrations indicateurs

In [ ]:
for pays in data_faostat['Pays'].unique() :
    data = data_faostat.loc[data_faostat['Indicateur']=='suffisance des apports énergétiques alimentaires moyens']
    data_pays = data.loc[data['Pays']==pays]
    plt.plot(data_pays['Année'],data_pays['Valeur'],label=pays)

plt.xlabel('Année')    
plt.ylabel('suffisance des apports énergétiques alimentaires moyens')
plt.grid()
plt.legend()
plt.xlim(left=2000, right=2025)
plt.show()

In [ ]:
for pays in data_faostat['Pays'].unique() :
    data = data_faostat.loc[data_faostat['Indicateur']=='disponibilité alimentaire par habitant']
    data_pays = data.loc[data['Pays']==pays]
    plt.plot(data_pays['Année'],data_pays['Valeur'],label=pays)

plt.xlabel('Année')    
plt.ylabel('disponibilité alimentaire par habitant')
plt.grid()
plt.legend()
plt.xlim(left=2000, right=2025)
plt.show()

In [ ]:
for pays in data_faostat['Pays'].unique() :
    data = data_faostat.loc[data_faostat['Indicateur']=='disponibilité protéiques moyenne']
    data_pays = data.loc[data['Pays']==pays]
    plt.plot(data_pays['Année'],data_pays['Valeur'],label=pays)

plt.xlabel('Année')    
plt.ylabel('disponibilité protéiques moyenne')
plt.grid()
plt.legend()
plt.xlim(left=2000, right=2025)
plt.show()

## Partie 3 : MODELE

### ACP

### Régression(s) linéaire PIB/hab

### Régression(s) linéaire IDH